1. https://www.aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=619 접속 후 데이터 중 라벨링데이터만 다운로드 받아 압축 해제
2. 현재 위치에 data 폴더 생성 후 안에 train_original, valid_original 라는 이름으로 저장

In [1]:
import csv
import json
import pandas as pd
from glob import glob
from tqdm import tqdm

In [2]:
def create_csv(target: str, half: bool = False):
    print(f"--- create {target} data (half={half}) ---")
    
    if half:
        json_paths = sorted(glob(f"./data/{target}_original/*/*.json", recursive=True))[:9]
        csv_path = f"./data/{target}_half.csv"
    else:
        json_paths = sorted(glob(f"./data/{target}_original/*/*.json", recursive=True))
        csv_path = f"./data/{target}.csv"

    # Write headers
    with open(csv_path, "w") as f:
        writer = csv.writer(f)
        writer.writerow(["text", "label"])

    # Wirte rows
    for idx, json_path in tqdm(enumerate(json_paths)):
        with open(json_path, "r") as f:
            json_dict = json.load(f)
        with open(csv_path, "a") as f:
            writer = csv.writer(f)
            for doc in json_dict["documents"]:
                sequence = doc["Q_refined"].replace("\n", "").strip()
                if sequence:
                    writer.writerow([sequence, idx])

    df = pd.read_csv(csv_path)
    print(f"Data size: {len(df)}")

In [3]:
def create_id_to_label(half: bool = False):
    print(f"--- create id to label (half={half}) ---")

    if half:
        dirs = sorted(glob(f"./data/train_original/*"))[:9]
        csv_path = "./data/id_to_label_half.csv"
    else:
        dirs = sorted(glob(f"./data/train_original/*"))
        csv_path = "./data/id_to_label.csv"

    labels = [dir.split("/")[-1] for dir in dirs]

    with open(csv_path, "w") as f:
        writer = csv.writer(f)
        writer.writerow(["id", "label"])
        for id, label in tqdm(enumerate(labels)):
            writer.writerow([id, label])

In [4]:
# Create id_to_label.csv
create_id_to_label()
create_id_to_label(half=True)

# Create train.csv
create_csv(target="train")
create_csv(target="train", half=True)

# Create valid.csv
create_csv(target="valid")
create_csv(target="valid", half=True)

--- create id to label (half=False) ---


18it [00:00, 193583.26it/s]


--- create id to label (half=True) ---


9it [00:00, 346318.68it/s]


--- create train data (half=False) ---


...

Data size: 285532
--- create valid data (half=False) ---


18it [00:01, 16.67it/s]


Data size: 100000
--- create valid data (half=True) ---


9it [00:00, 24.19it/s]

Data size: 35692
